# CMG Online Endpoint Testing
Testing different strategies to reliably fetch Chiloé data from CMG Online

In [1]:
import requests
import time
from datetime import datetime, timedelta
import json
import pytz

# Configuration
SIP_API_KEY = '1a81177c8ff4f69e7dd5bb8c61bc08b4'
SIP_BASE_URL = 'https://sipub.api.coordinador.cl:443'
CHILOE_NODE = 'CHILOE________220'

santiago_tz = pytz.timezone('America/Santiago')
now = datetime.now(santiago_tz)
print(f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S %Z')}")

Current time: 2025-08-25 19:49:12 -04


## Test 1: Direct API Call with Small Limit

In [2]:
def test_cmg_online(date_str, page=1, limit=100):
    """Test CMG Online endpoint with specific parameters"""
    url = f"{SIP_BASE_URL}/costo-marginal-online/v4/findByDate"
    
    params = {
        'startDate': date_str,
        'endDate': date_str,
        'page': page,
        'limit': limit,
        'user_key': SIP_API_KEY
    }
    
    print(f"\nTesting: {date_str}, page {page}, limit {limit}")
    print(f"URL: {url}")
    print(f"Params: {params}")
    
    start_time = time.time()
    
    try:
        response = requests.get(url, params=params, timeout=30)
        elapsed = time.time() - start_time
        
        print(f"Response: {response.status_code} in {elapsed:.2f}s")
        
        if response.status_code == 200:
            data = response.json()
            
            # Check structure
            total_pages = data.get('totalPages', 0)
            page_num = data.get('page', 1)
            limit_num = data.get('limit', limit)
            records = data.get('data', [])
            
            print(f"Total pages: {total_pages}")
            print(f"Current page: {page_num}")
            print(f"Records in this page: {len(records)}")
            
            # Look for Chiloé
            chiloe_found = False
            for record in records:
                if record.get('barra_transf') == CHILOE_NODE:
                    chiloe_found = True
                    print(f"✅ FOUND CHILOÉ: {record['fecha_hora']} - ${record.get('cmg_usd_mwh_', 0):.2f}")
            
            if not chiloe_found:
                # Show sample nodes
                sample_nodes = list(set(r.get('barra_transf', '') for r in records[:10]))
                print(f"Sample nodes: {sample_nodes[:5]}")
            
            return data
        else:
            print(f"Error response: {response.text[:200]}")
            return None
            
    except requests.exceptions.Timeout:
        print(f"❌ Timeout after 30s")
        return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Test today with small limit
today = now.strftime('%Y-%m-%d')
result = test_cmg_online(today, page=1, limit=100)


Testing: 2025-08-25, page 1, limit 100
URL: https://sipub.api.coordinador.cl:443/costo-marginal-online/v4/findByDate
Params: {'startDate': '2025-08-25', 'endDate': '2025-08-25', 'page': 1, 'limit': 100, 'user_key': '1a81177c8ff4f69e7dd5bb8c61bc08b4'}
Response: 200 in 8.69s
Total pages: 1074
Current page: 1
Records in this page: 100
Sample nodes: ['M.DEVELASCO___013', 'SAUZAL________013', 'SANTA_ROSA_S__110', 'LOTA__________066', 'SAUZAL2_______110']


## Test 2: Binary Search for Chiloé Node

In [3]:
def find_chiloe_binary_search(date_str, max_pages=10):
    """Use binary search to find which page contains Chiloé"""
    print(f"\nSearching for Chiloé node on {date_str}...")
    
    # First, get total pages with minimal data
    url = f"{SIP_BASE_URL}/costo-marginal-online/v4/findByDate"
    params = {
        'startDate': date_str,
        'endDate': date_str,
        'page': 1,
        'limit': 1,  # Just to get total pages
        'user_key': SIP_API_KEY
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            total_pages = response.json().get('totalPages', 0)
            print(f"Total pages available: {total_pages}")
        else:
            print(f"Failed to get total pages: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error getting total pages: {e}")
        return None
    
    # Now search pages systematically
    chiloe_data = []
    pages_to_check = list(range(1, min(max_pages + 1, total_pages + 1)))
    
    # Try different page sizes
    for limit in [500, 250, 100]:
        print(f"\nTrying with limit={limit}...")
        
        for page in pages_to_check:
            params = {
                'startDate': date_str,
                'endDate': date_str,
                'page': page,
                'limit': limit,
                'user_key': SIP_API_KEY
            }
            
            print(f"  Checking page {page}...", end='')
            
            try:
                response = requests.get(url, params=params, timeout=20)
                
                if response.status_code == 200:
                    data = response.json()
                    records = data.get('data', [])
                    
                    # Check for Chiloé
                    for record in records:
                        if record.get('barra_transf') == CHILOE_NODE:
                            chiloe_data.append({
                                'datetime': record['fecha_hora'],
                                'cmg': record.get('cmg_usd_mwh_', 0),
                                'page': page,
                                'limit': limit
                            })
                    
                    if chiloe_data:
                        print(f" ✅ Found {len(chiloe_data)} Chiloé records!")
                        return chiloe_data
                    else:
                        print(f" No Chiloé")
                        
                elif response.status_code == 429:
                    print(f" Rate limited, waiting...")
                    time.sleep(3)
                elif response.status_code == 500:
                    print(f" Server error")
                    break  # Try smaller limit
                else:
                    print(f" Status {response.status_code}")
                    
            except requests.exceptions.Timeout:
                print(f" Timeout")
                break  # Try smaller limit
            except Exception as e:
                print(f" Error: {e}")
                break
        
        if chiloe_data:
            break
        
        # Wait before trying smaller limit
        if limit > 100:
            print("Waiting before retry with smaller limit...")
            time.sleep(2)
    
    return chiloe_data

# Search for Chiloé
chiloe_records = find_chiloe_binary_search(today)


Searching for Chiloé node on 2025-08-25...
Total pages available: 112109

Trying with limit=500...
  Checking page 1... No Chiloé
  Checking page 2... No Chiloé
  Checking page 3... ✅ Found 1 Chiloé records!


## Test 3: Fetch Multiple Days with Retry Logic

In [4]:
def fetch_with_smart_retry(url, params, max_retries=3, initial_wait=1):
    """Fetch with exponential backoff"""
    wait_time = initial_wait
    
    for attempt in range(max_retries):
        try:
            print(f"    Attempt {attempt + 1}/{max_retries}...", end='')
            response = requests.get(url, params=params, timeout=15)
            
            if response.status_code == 200:
                print(" ✅")
                return response.json()
            elif response.status_code in [429, 500, 502, 503]:
                if attempt < max_retries - 1:
                    print(f" {response.status_code}, waiting {wait_time}s...")
                    time.sleep(wait_time)
                    wait_time = min(wait_time * 2, 10)
                else:
                    print(f" {response.status_code} - giving up")
            else:
                print(f" {response.status_code}")
                return None
                
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                print(f" Timeout, waiting {wait_time}s...")
                time.sleep(wait_time)
                wait_time = min(wait_time * 2, 10)
            else:
                print(" Timeout - giving up")
        except Exception as e:
            print(f" Error: {e}")
            return None
    
    return None

def fetch_last_48h_cmg():
    """Fetch last 48 hours of CMG data"""
    yesterday = now - timedelta(days=1)
    day_before = now - timedelta(days=2)
    
    all_data = {}
    
    print(f"\nFetching last 48 hours of CMG data...")
    print(f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}")
    
    dates = [
        (day_before, "Day before yesterday"),
        (yesterday, "Yesterday"),
        (now, "Today")
    ]
    
    url = f"{SIP_BASE_URL}/costo-marginal-online/v4/findByDate"
    
    for date, label in dates:
        date_str = date.strftime('%Y-%m-%d')
        print(f"\n{label} ({date_str}):")
        
        # Try with different limits
        for limit in [100, 250]:
            print(f"  Trying limit={limit}...")
            
            found_any = False
            for page in range(1, 6):  # Check first 5 pages
                params = {
                    'startDate': date_str,
                    'endDate': date_str,
                    'page': page,
                    'limit': limit,
                    'user_key': SIP_API_KEY
                }
                
                print(f"  Page {page}: ", end='')
                data = fetch_with_smart_retry(url, params)
                
                if data and 'data' in data:
                    records = data.get('data', [])
                    
                    for record in records:
                        if record.get('barra_transf') == CHILOE_NODE:
                            dt_key = record['fecha_hora'][:16]
                            all_data[dt_key] = {
                                'datetime': dt_key,
                                'cmg': float(record.get('cmg_usd_mwh_', 0))
                            }
                            found_any = True
                    
                    if found_any:
                        print(f"    Found Chiloé data!")
                        break
                
                if not data:
                    break  # API failed, try smaller limit
            
            if found_any:
                break  # Found data, move to next date
    
    # Summary
    sorted_data = sorted(all_data.values(), key=lambda x: x['datetime'])
    print(f"\n📊 SUMMARY:")
    print(f"  Total points: {len(sorted_data)}")
    if sorted_data:
        print(f"  Date range: {sorted_data[0]['datetime']} to {sorted_data[-1]['datetime']}")
        print(f"\n  Last 5 values:")
        for item in sorted_data[-5:]:
            print(f"    {item['datetime']}: ${item['cmg']:.2f}")
    
    return sorted_data

# Fetch data
cmg_data = fetch_last_48h_cmg()


Fetching last 48 hours of CMG data...
Current time: 2025-08-25 19:49:12

Day before yesterday (2025-08-23):
  Trying limit=100...
  Page 1:     Attempt 1/3... ✅
  Page 2:     Attempt 1/3... ✅
  Page 3:     Attempt 1/3... ✅
  Page 4:     Attempt 1/3... ✅
  Page 5:     Attempt 1/3... ✅
  Trying limit=250...
  Page 1:     Attempt 1/3... ✅
  Page 2:     Attempt 1/3... ✅
  Page 3:     Attempt 1/3... ✅
  Page 4:     Attempt 1/3... ✅
  Page 5:     Attempt 1/3... ✅

Yesterday (2025-08-24):
  Trying limit=100...
  Page 1:     Attempt 1/3... ✅
  Page 2:     Attempt 1/3... ✅
  Page 3:     Attempt 1/3... ✅
  Page 4:     Attempt 1/3... ✅
    Found Chiloé data!

Today (2025-08-25):
  Trying limit=100...
  Page 1:     Attempt 1/3... ✅
  Page 2:     Attempt 1/3... ✅
  Page 3:     Attempt 1/3... ✅
  Page 4:     Attempt 1/3... ✅
  Page 5:     Attempt 1/3... ✅
  Trying limit=250...
  Page 1:     Attempt 1/3... ✅
  Page 2:     Attempt 1/3... ✅
  Page 3:     Attempt 1/3... ✅
  Page 4:     Attempt 1/3... ✅

## Test 4: Check Node Names Pattern

In [5]:
def analyze_node_names(date_str):
    """Analyze node naming patterns to find Chiloé"""
    print(f"\nAnalyzing node names for {date_str}...")
    
    url = f"{SIP_BASE_URL}/costo-marginal-online/v4/findByDate"
    params = {
        'startDate': date_str,
        'endDate': date_str,
        'page': 1,
        'limit': 1000,  # Get many nodes
        'user_key': SIP_API_KEY
    }
    
    try:
        print("Fetching first 1000 nodes...")
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            records = data.get('data', [])
            
            # Get all unique nodes
            all_nodes = list(set(r.get('barra_transf', '') for r in records))
            all_nodes.sort()
            
            print(f"Total unique nodes in first 1000: {len(all_nodes)}")
            
            # Look for Chiloé-related
            chiloe_related = [n for n in all_nodes if 'CHILO' in n.upper()]
            print(f"\nChiloé-related nodes found:")
            for node in chiloe_related:
                print(f"  - {node}")
            
            # Show nodes starting with 'C'
            c_nodes = [n for n in all_nodes if n.startswith('C')]
            print(f"\nNodes starting with 'C' (first 10):")
            for node in c_nodes[:10]:
                print(f"  - {node}")
            
            return all_nodes
        else:
            print(f"Failed: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []

# Analyze nodes
nodes = analyze_node_names(today)


Analyzing node names for 2025-08-25...
Fetching first 1000 nodes...
Total unique nodes in first 1000: 524

Chiloé-related nodes found:

Nodes starting with 'C' (first 10):
  - C.COMBARBALA__023
  - C.COMBARBALA__110
  - C.CONSTIT1____023
  - C.SAN_ANDRES__023
  - C.SAN_ANDRES__220
  - CAMPO_LINDO1__033
  - CAMPO_LINDO2__033
  - CAMPO_LINDO___220
  - CANTERAS______023
  - CANTERAS______110


## Test 5: Optimized Fetching Strategy

In [6]:
def optimized_cmg_fetch():
    """Optimized strategy for fetching CMG Online data"""
    
    print("\n" + "="*50)
    print("OPTIMIZED CMG ONLINE FETCH")
    print("="*50)
    
    all_chiloe_data = {}
    url = f"{SIP_BASE_URL}/costo-marginal-online/v4/findByDate"
    
    # Dates to fetch
    dates = [
        now - timedelta(days=2),
        now - timedelta(days=1),
        now
    ]
    
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        print(f"\n📅 {date_str}:")
        
        # Strategy: Start with small pages, increase if successful
        limits = [50, 100, 200]
        
        for limit in limits:
            print(f"  Trying limit={limit}...")
            
            success_count = 0
            fail_count = 0
            
            for page in range(1, 11):  # Check up to 10 pages
                params = {
                    'startDate': date_str,
                    'endDate': date_str,
                    'page': page,
                    'limit': limit,
                    'user_key': SIP_API_KEY
                }
                
                try:
                    start = time.time()
                    response = requests.get(url, params=params, timeout=10)
                    elapsed = time.time() - start
                    
                    if response.status_code == 200:
                        success_count += 1
                        data = response.json()
                        
                        # Look for Chiloé
                        found_in_page = False
                        for record in data.get('data', []):
                            if record.get('barra_transf') == CHILOE_NODE:
                                dt_key = record['fecha_hora'][:16]
                                cmg = float(record.get('cmg_usd_mwh_', 0))
                                
                                all_chiloe_data[dt_key] = {
                                    'datetime': dt_key,
                                    'cmg': cmg
                                }
                                found_in_page = True
                        
                        if found_in_page:
                            print(f"    Page {page}: ✅ Found Chiloé ({elapsed:.1f}s)")
                            break  # Found for this date, move on
                        else:
                            print(f"    Page {page}: No Chiloé ({elapsed:.1f}s)")
                    
                    elif response.status_code in [429, 500, 502, 503]:
                        fail_count += 1
                        print(f"    Page {page}: {response.status_code} - retrying...")
                        time.sleep(2 * fail_count)  # Increasing backoff
                        
                        if fail_count >= 3:
                            print(f"    Too many failures, trying smaller limit")
                            break
                    else:
                        print(f"    Page {page}: Status {response.status_code}")
                        break
                        
                except requests.exceptions.Timeout:
                    print(f"    Page {page}: Timeout")
                    fail_count += 1
                    if fail_count >= 2:
                        break
                except Exception as e:
                    print(f"    Page {page}: Error - {e}")
                    break
            
            # If we found data with this limit, move to next date
            date_data = [v for k, v in all_chiloe_data.items() if k.startswith(date_str)]
            if date_data:
                print(f"  ✅ Got {len(date_data)} records for {date_str}")
                break
    
    # Final summary
    sorted_data = sorted(all_chiloe_data.values(), key=lambda x: x['datetime'])
    
    print("\n" + "="*50)
    print("FINAL RESULTS:")
    print(f"Total Chiloé records: {len(sorted_data)}")
    
    if sorted_data:
        print(f"Date range: {sorted_data[0]['datetime']} to {sorted_data[-1]['datetime']}")
        
        # Group by date
        by_date = {}
        for item in sorted_data:
            date_key = item['datetime'][:10]
            if date_key not in by_date:
                by_date[date_key] = []
            by_date[date_key].append(item)
        
        print("\nRecords by date:")
        for date_key in sorted(by_date.keys()):
            records = by_date[date_key]
            avg_cmg = sum(r['cmg'] for r in records) / len(records)
            print(f"  {date_key}: {len(records)} records, avg ${avg_cmg:.2f}")
        
        print("\nLast 5 records:")
        for item in sorted_data[-5:]:
            print(f"  {item['datetime']}: ${item['cmg']:.2f}")
    
    return sorted_data

# Run optimized fetch
final_data = optimized_cmg_fetch()


OPTIMIZED CMG ONLINE FETCH

📅 2025-08-23:
  Trying limit=50...
    Page 1: Timeout
    Page 2: Timeout
  Trying limit=100...
    Page 1: Timeout
    Page 2: Timeout
  Trying limit=200...
    Page 1: Timeout
    Page 2: Timeout

📅 2025-08-24:
  Trying limit=50...
    Page 1: Timeout
    Page 2: Timeout
  Trying limit=100...
    Page 1: Timeout
    Page 2: Timeout
  Trying limit=200...
    Page 1: Timeout
    Page 2: Timeout

📅 2025-08-25:
  Trying limit=50...
    Page 1: No Chiloé (9.8s)
    Page 2: No Chiloé (9.0s)
    Page 3: No Chiloé (9.7s)
    Page 4: Timeout


KeyboardInterrupt: 